In [1]:
from azureml.core import Workspace
from azureml.core import Experiment
from azureml.core import Dataset
from azureml.core.compute import AmlCompute, ComputeTarget
from azureml.core import Environment
from azureml.core.conda_dependencies import CondaDependencies
from azureml.core import ScriptRunConfig
from azureml.core import Model

from azureml.exceptions import ComputeTargetException

# own modules
import sys
sys.path.append('../')
from config.config_training import script_params_remote

In [2]:
# important parameters
compute_name = 'dp-100-v2'
project_folder = '../'

# 1. Configure Workspace

In [3]:
ws = Workspace.from_config('../config/config.json')

# 2. Configure Compute Target

In [4]:
try:
    comput_target = ComputeTarget(workspace=ws, name= compute_name)
    print('Comput target exists!')
except:
    config = AmlCompute.provisioning_configuration(vm_size='STANDARD_DS11_VS2'
                                                          ,vm_priority='lowpriority'
                                                          ,min_nodes=0
                                                          , max_nodes=1)
    comput_target = ComputeTarget.create(workspace=ws, name=compute_name, provisioning_configuration=config)
    compute_target.wait_for_completion(show_output=True, min_node_count=None, timeout_in_minutes=20)                                       

Comput target exists!


# 3. Configurate Dependencies

In [4]:
env = Environment('user-managed-env')
env.python.user_managed_dependencies = True

# 4. Load Data

Run this in conda shell before starting jupyter lab: __export LC_ALL=en_US.UTF-8__

In [5]:
tab_ds = Dataset.get_by_name(ws, 'diabetes_dataset')
script_params_remote.extend(['--ds', tab_ds.as_named_input('diabetes_dataset')])

# 5. Configure Experiment

In [6]:
src = ScriptRunConfig(source_directory=project_folder
                     ,script='./src/train.py'
                     ,arguments=script_params_remote)

In [7]:
src.run_config.target = 'local'
src.run_config.environment = env

In [8]:
experiment = Experiment(workspace=ws, name='test_v5')

# 6. Run Experiment und Monitor Training

In [9]:
run = experiment.submit(src)
run.wait_for_completion(show_output=True)

RunId: test_v5_1612126669_08aad9a2
Web View: https://ml.azure.com/experiments/test_v5/runs/test_v5_1612126669_08aad9a2?wsid=/subscriptions/c2cbd114-566e-42ab-a9f3-ae8000df45a0/resourcegroups/certification_dp100/workspaces/dp_100_v2

Streaming azureml-logs/60_control_log.txt

[2021-01-31T20:57:51.893440] Entering context manager injector.
[2021-01-31T20:57:53.712166] Using urllib.request Python 3.0 or later
Streaming log file azureml-logs/60_control_log.txt
Starting the daemon thread to refresh tokens in background for process with pid = 9349
Running: ['/bin/bash', '/private/var/folders/2x/5phlmdsd0td5pb7srj3fpqth0000gn/T/azureml_runs/test_v5_1612126669_08aad9a2/azureml-environment-setup/conda_env_checker.sh']


Running: ['python', 'azureml-setup/run_script.py', 'python', 'azureml-setup/context_manager_injector.py', '-i', 'ProjectPythonPath:context_managers.ProjectPythonPath', '-i', 'RunHistory:context_managers.RunHistory', '-i', 'TrackUserError:context_managers.TrackUserError', 'src/tr

{'runId': 'test_v5_1612126669_08aad9a2',
 'target': 'local',
 'status': 'Completed',
 'startTimeUtc': '2021-01-31T20:57:53.823098Z',
 'endTimeUtc': '2021-01-31T20:59:02.558614Z',
 'properties': {'_azureml.ComputeTargetType': 'local',
  'ContentSnapshotId': 'b9a001a3-1dca-4831-93c6-2d74a41e9a39',
  'azureml.git.repository_uri': 'https://github.com/carlov93/azure_dp_100.git',
  'mlflow.source.git.repoURL': 'https://github.com/carlov93/azure_dp_100.git',
  'azureml.git.branch': 'master',
  'mlflow.source.git.branch': 'master',
  'azureml.git.commit': '1edba43d0b6e436e69badb662a9744b9172d64db',
  'mlflow.source.git.commit': '1edba43d0b6e436e69badb662a9744b9172d64db',
  'azureml.git.dirty': 'True'},
 'inputDatasets': [{'dataset': {'id': '6c2f466e-bfb8-4b2e-ba13-c210f564d58b'}, 'consumptionDetails': {'type': 'RunInput', 'inputName': 'diabetes_dataset', 'mechanism': 'Direct'}}],
 'outputDatasets': [],
 'runDefinition': {'script': 'src/train.py',
  'command': '',
  'useAbsolutePath': False,
  

# 7. Add trained motel to workspace

In [12]:
model = Model.register(workspace=ws
                       ,model_path='../trained_models/diabetes_model.pkl'
                       ,model_name='Diabetes_Classifier'
                       ,description='First trained model with Azure ML'
                       ,tags={'data-format': 'csv'}
                       ,properties={'Accuracy': run.get_metrics()['Accuracy']}
                       ,model_framework=Model.Framework.SCIKITLEARN  # Framework used to create the model.
                       ,model_framework_version='0.23.2'
                       ,datasets=[('training_data', tab_ds)]
                       )

KeyError: 'Accuracy'